In [2]:
# ******* Flags and constants space  *******
inputDBG=0  # flag for debugging the takeInput Function
inputTest=0 # flag used to test the input
calcScoreTest=0 # flag used to test the score calc fn
calcScoreDBG=0  # flag for debugging the calcScore Function
greedyTest=0    # flag used to test the greedy fn
greedyDBG=0        # flag for debugging the greedy Function
calcOverallScore=1 # test used for calculate every 
greedy_time_dbg=1  # flag used to test the time taken from the greedy


# indexes of the library descriptor lists  
numBooksIdx=0
signupTimeIdx=1
booksPerDayIdx=2
# imports
if greedy_time_dbg==1:
    import time as time


In [3]:
# ******* Functions space  *******

In [4]:
# This function reads a file in fileLocation in order to generate a problem instance 
# This function returns a tuple: (bookScores,libraryData,libraryBooks,numDays)
# bookScores -> list containing the score of the i-th book
# libraryData-> list of lists, each single list contains the (num books, signup, books per day) of the i-th lib
# libraryBooks-> list of lists, each single list is the list of books of the i-th lib
# numDays-> the number of days ( i.e. 7 days , then , counting from day 0, last day  is 7-1 = 6 )
def takeInput(fileLocation):
    numBooks=0 
    numLibs=0
    numDays=0
    bookScores=[]  # no need for a map, just a list
    libraryData=[] # List of lists, list structure:  ( number of Books In The Lib, Signup days, books shipped per day)
    libraryBooks=[] # List of lists, each list contains the books in a specific library
    with open(fileLocation,'r') as file:
        numBooks,numLibs,numDays = [int(i) for i in file.readline().split()] # take numbers
        bookScores=file.readline().split() # take book scores
        bookScores = [int(i) for i in bookScores] # convert them to integers
        for i in range(numLibs):# for each library
            # Reading single lib data
            libraryData.append(file.readline().split()) 
            libraryData[i]= [int(j) for j in libraryData[i]] # convert the single list to int
            # reading books data
            libraryBooks.append(file.readline().split())
            libraryBooks[i]=[int(j) for j in libraryBooks[i]]
        
    if inputDBG==1:
        print(numBooks ,    numLibs, numDays)
        for i in range(len(bookScores)):
            print( " Book id ", i, " Score ", bookScores[i], type(bookScores[i]))
        for i in range(numLibs):
            print(" For lib ",i, "  number of books  ", libraryData[i][0], " Signup time ",libraryData[i][1], "Cap",libraryData[i][2] )
            print ("books ", libraryBooks[i])
        
    return (bookScores,libraryData,libraryBooks,numDays)

In [13]:
# Writest the output on a file
# librariesOrdered-> libraries in order
# booksPerLib     -> books for each lib
def writeOut(fileLocation,librariesOrdered,booksPerLib):
     with open(fileLocation,'w') as file:
            file.write(str(len(librariesOrdered))+"\n")
            for lib in librariesOrdered:   
                file.write(str(lib)+" "+str(len(booksPerLib[lib]))+"\n")
                for book in booksPerLib[lib]:
                    file.write( str(book)+" ")
                file.write("\n")

In [6]:
# Function used to calculate the  score of a library in a specific days given its signup time and capacity ( books per day)
# Inputs:
# Actual day -> the day we are in
# Last day   -> the last usefull day for book sending
# libCapacity-> Number of books per day sended from the lib
# libSignup  -> time used for  signing up
# booksScore -> book score list
# lk         -> books already scheduled list 
# Returns    ':'(score for greedy choice, score, numBooksToSend)
#O(nlogn+n)=O(nlogn) n, num books in the lib
def calcScore(actualDay,lastDay,libCapacity,libSignup,libBooks,bookScores,lk):
    
    # init the score
    score=0
    
    # Sort the books to send, O(nlog(n)), number of books
    libBooks.sort( key=lambda x: bookScores[x],reverse=True)
    
    # calculate the number of books to send
    numBooksToSend=min(len(libBooks),libCapacity*(lastDay-(actualDay+libSignup)) )
    
     # can be <0 due to deadline overflow
    if numBooksToSend<0:
        numBooksToSend=0
        
    if calcScoreDBG==1:
        print ("****** BEGIN CALC SCORE DBG **********")
        print ("Days to send ",(lastDay-(actualDay+libSignup)))
        print(" Maximum books ",(lastDay-(actualDay+libSignup))*libCapacity)
        print(" Number of books to send ",numBooksToSend)
        print ("****** END CALC SCORE DBG **********")
    
    # calculate the score, O(n)
    (i,books_placed)=(0,0)
    
    #untill we've ended the list of the books_placed
    #O(n)
    while i<len(libBooks) and books_placed<numBooksToSend:
        # if the book was already sended
        if lk[libBooks[i]]:
            i+=1
            #print("book already in list ", libBooks[i])
        else:
            # in case the book was not sent increase the score
            score+=bookScores[libBooks[i]]
            # increase the number of books placed
            i+=1
            books_placed+=1
   
    return (score/libSignup,score,numBooksToSend)

In [7]:
# Greedy algo for problem solve
# libraryData -> the data of each lib
# libraryBooks-> the books for each lib
# bookScores  -> the score of each book
# numDays     -> the number of days 
def greedy(libraryData,libraryBooks,bookScores,numDays):
    # time debug
    if greedy_time_dbg:
        greedy_time=time.time()
    
    # time parameters
    actualDay=0
    lastDay=numDays-1
    
    # score
    TotalScore=0
    
    # books per lib map
    booksPerLib={}
    # libraries
    librariesOrdered=[]

    (libCount,libCountUB)=(0,len(libraryData))
    #libs scheduled , 0-numLibs-1, libsScheduled[i] if a lib has already signed up
    libsScheduled=[0]*libCountUB
    #Initialize the score
    (actualScore,prevScore)=(0,0)
    #books scheduled , 0-numBooks-1, booksScheduled[i] if a lib will send the book
    booksScheduled=[0]*len(bookScores)
    
    # helper score used during iterations
    actualScore=0
    # idx of the best scoring lib, during iterations
    bestScoringIdx=0
    # Scoring of the best book
    bestScoringNumBooks=0
    # score of the best library ( this is not the score for the greedy solution)
    toAddScore=0
    
    #Until the deadline
    #O(m(nlogn+n)), m num libs, n num books
    while actualDay<numDays:
        
        # if no libraries
        if libCount==libCountUB:
            break
        
        # find the best lib
        actualScore=0
        
        # for each non scheduled library
        for lib in range(libCountUB):
            if not libsScheduled[lib]:
                (newScore,add_sc,numBooks)=calcScore(actualDay,lastDay,libraryData[lib][booksPerDayIdx],
                                libraryData[lib][signupTimeIdx],libraryBooks[lib],bookScores,booksScheduled)
                 # in case it is better update
                if actualScore<newScore:
                    actualScore=newScore
                    bestScoringIdx=lib # value== idx
                    bestScoringNumBooks=numBooks
                    toAddScore=add_sc
        
        # can't be improved
        if actualScore==0:  
            break 

        # increase the num of found libs
        libCount+=1        
        # remove the element from list
        libsScheduled[bestScoringIdx]=1
        # update the time
        actualDay+=libraryData[bestScoringIdx][signupTimeIdx]
        # update the score
        TotalScore+=toAddScore

        # Find  the books sended O(n)
        (i,books_placed)=(0,0)
        #O(n)
        booksPerLib[bestScoringIdx]=[]
        #i<libraryData[bestScoringIdx][numBooksIdx]
        while  i<libraryData[bestScoringIdx][numBooksIdx] and books_placed<bestScoringNumBooks:
            if booksScheduled[libraryBooks[bestScoringIdx][i]]:
                i+=1
            else:
                booksScheduled[libraryBooks[bestScoringIdx][i]]=1 
                booksPerLib[bestScoringIdx].append(libraryBooks[bestScoringIdx][i])
                i+=1
                books_placed+=1
                
        librariesOrdered.append(bestScoringIdx) 
    if greedy_time_dbg:
        greedy_time=time.time()-greedy_time
        print ("Time Taken" ,greedy_time)
    return (TotalScore,librariesOrdered,booksPerLib )

In [8]:
# *** Tests***
# Input
if inputTest==1:
    (bookScores,libraryData,libraryBooks,numDays)=takeInput("./Inputs/a_example.txt")
    print("days " , numDays)
    for i in range(len(bookScores)):
        print( " Book id ", i, " Score ", bookScores[i], type(bookScores[i]))
    for i in range(len(libraryData)):
        print(" For lib ",i, "  number of books  ", libraryData[i][0], " Signup time ",libraryData[i][1], "Cap",libraryData[i][2] )
        print ("books ", libraryBooks[i])
# Scoring
if calcScoreTest==1:
    (bookScores,libraryData,libraryBooks,numDays)=takeInput("./Inputs/a_example.txt")
    lk=[0]*len(bookScores)
    print ("lk",lk)
    print(" ******* DAY 0 ********")
    # test on day 0
    for i in range( len(libraryData)):
        (score, books)=calcScore(0,numDays-1,libraryData[i][booksPerDayIdx], libraryData[i][signupTimeIdx],
                                 libraryBooks[i],bookScores,lk)
        print("Lib ",i, " Score ",score," Books", books)
    # test on day 1
    print(" ******* DAY 1 ********")
    for i in range( len(libraryData)):
        (score, books)=calcScore(1,numDays-1,libraryData[i][booksPerDayIdx], 
                                 libraryData[i][signupTimeIdx],libraryBooks[i],bookScores,lk)
        print("Lib ",i, " Score ",score," Books", books)
    # test on 2 days left
    print(" *********. Two days left ********")
    for i in range( len(libraryData)):
       
        (score, books)=calcScore(numDays-3,numDays-1,libraryData[i][booksPerDayIdx],
                                 libraryData[i][signupTimeIdx],libraryBooks[i],bookScores,lk)
        print("Lib ",i, " Score ",score," Books", books)
        
if greedyTest:
    (TotalScore,librariesOrdered,booksPerLib )=greedy(libraryData,libraryBooks,bookScores,numDays)
    print(TotalScore, librariesOrdered,booksPerLib)
    
        

In [14]:
if calcOverallScore:
    #scores= [] # scores of each problem
    #librariesOrdered =[[]] # library schedule for each problem
    #librariesBooks=[[[]]] # books for each lib for each problem
    total_score=0
    
    # example
    (bookScores,libraryData,libraryBooks,numDays)=takeInput("./Inputs/a_example.txt")
    (TotalScore,librariesOrdered,booksPerLib )=greedy(libraryData,libraryBooks,bookScores,numDays)
    print ( " For problem example the score is : ",TotalScore)
    writeOut("./Outs/a_example_out.txt",librariesOrdered,booksPerLib)
    total_score+=TotalScore
    
    # b_readon
    (bookScores,libraryData,libraryBooks,numDays)=takeInput("./Inputs/b_read_on.txt")
    (TotalScore,librariesOrdered,booksPerLib )=greedy(libraryData,libraryBooks,bookScores,numDays)
    print ( " For problem b the score is : ",TotalScore)
    writeOut("./Outs/b_read_on_out.txt",librariesOrdered,booksPerLib)
    total_score+=TotalScore
    
    # c_incunabula
    (bookScores,libraryData,libraryBooks,numDays)=takeInput("./Inputs/c_incunabula.txt")
    (TotalScore,librariesOrdered,booksPerLib )=greedy(libraryData,libraryBooks,bookScores,numDays)
    print ( " For problem c the score is : ",TotalScore)
    writeOut("./Outs/c_incunabula_out.txt",librariesOrdered,booksPerLib)
    total_score+=TotalScore
    
    # e_so_many_books
    (bookScores,libraryData,libraryBooks,numDays)=takeInput("./Inputs/e_so_many_books.txt")
    (TotalScore,librariesOrdered,booksPerLib )=greedy(libraryData,libraryBooks,bookScores,numDays)
    print ( " For problem e the score is : ",TotalScore)
    writeOut("./Outs/e_so_many_books_out.txt",librariesOrdered,booksPerLib)
    total_score+=TotalScore
    
        
    # f_libraries_of_the_world
    (bookScores,libraryData,libraryBooks,numDays)=takeInput("./Inputs/f_libraries_of_the_world.txt")
    (TotalScore,librariesOrdered,booksPerLib )=greedy(libraryData,libraryBooks,bookScores,numDays)
    print ( " For the problem f  the score is : ",TotalScore)
    writeOut("./Outs/f_libraries_of_the_world_out.txt",librariesOrdered,booksPerLib)
    total_score+=TotalScore
    
    # d_though_choices
    (bookScores,libraryData,libraryBooks,numDays)=takeInput("./Inputs/d_tough_choices.txt")
    (TotalScore,librariesOrdered,booksPerLib )=greedy(libraryData,libraryBooks,bookScores,numDays)
    print ( " For problem d the score is : ",TotalScore)
    writeOut("./Outs/d_tough_choices_out.txt",librariesOrdered,booksPerLib)
    total_score+=TotalScore
    
    
    print( " In the end, the total score is ", total_score)

Time Taken 0.0
 For problem example the score is :  21
Time Taken 0.7396676540374756
 For problem b the score is :  5813900
Time Taken 37.57427191734314
 For problem c the score is :  5689822
Time Taken 7.095980167388916
 For problem e the score is :  5003279
Time Taken 1.511368989944458
 For the problem f  the score is :  5302835
Time Taken 529.0141208171844
 For problem d the score is :  5028465
 In the end, the total score is  26838322
